# Prompt 模板
（摘自LangChain中文文档）

提示模板有助于将用户输入和参数转换为语言模型的指令。这可以用于指导模型的响应，帮助其理解上下文并生成相关且连贯的基于语言的输出。

提示模板接收字典作为输入，其中每个键代表提示模板中要填充的变量。

提示模板输出一个 PromptValue。这个 PromptValue 可以传递给 LLM 或 ChatModel，也可以转换为字符串或消息列表。PromptValue 存在的原因是为了方便在字符串和消息之间切换。

提示模板有几种不同类型

## 字符串提示模板
建和使用 PromptTemplate 的常见方法如下

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})#似乎还用到了异步的思想？

print(prompt_template)

input_variables=['topic'] input_types={} partial_variables={} template='Tell me a joke about {topic}'


## 聊天提示模板
这些提示模板用于格式化消息列表。这些“模板”本身由一个模板列表组成。例如，构建和使用 ChatPromptTemplate 的常见方法如下

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

在上面的示例中，当调用此 ChatPromptTemplate 时，它将构建两条消息。第一条是系统消息，没有要格式化的变量。第二条是 HumanMessage，它将由用户传入的 topic 变量进行格式化。
## 消息占位符
希望用户传入一个消息列表，并将其放入指定的位置：

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

这将生成一个包含两条消息的列表，第一条是系统消息，第二条是我们传入的 HumanMessage。如果我们传入了 5 条消息，那么它总共会生成 6 条消息（系统消息加上传入的 5 条消息）。这对于将消息列表插入特定位置非常有用。

实现相同目标的另一种方法，不显式使用 MessagesPlaceholder 类，如下所示



In [ ]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{msgs}") # <-- This is the changed part，相比于前一段代码修改的内容
])

## 字符串提示词组合
两种方法：

In [4]:
from langchain_core.prompts import PromptTemplate
#法一
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)
#法二
prompt=PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

prompt.format(topic="sports", language="spanish")

print(prompt)

input_variables=['language', 'topic'] input_types={} partial_variables={} template='Tell me a joke about {topic}, make it funny\n\nand in {language}'


## 聊天提示词组合
聊天提示词模板也可以连接，每个新元素都是最终提示词中的一条新消息。

首先，让我们使用一个 ChatPromptTemplate 初始化一个 SystemMessage。

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

prompt = SystemMessage(content="You are a nice pirate")

然后，可以创建一个管道，将其与其他消息或消息模板结合。当没有变量需要格式化时，使用 Message；当有变量需要格式化时，使用 MessageTemplate。您也可以只使用字符串（注意：这将被自动推断为 HumanMessagePromptTemplate）。

In [ ]:
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)

在底层，这会创建一个 ChatPromptTemplate 类的实例。

In [ ]:
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

## 如何部分格式化提示模板
LangChain 以两种方式支持这一点

使用字符串值进行部分格式化。
使用返回字符串值的函数进行部分格式化。
### 使用字符串值进行部分格式化

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))

foobaz


In [7]:
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="baz"))#也可以直接用部分格式化够的变量初始化提示词

foobaz


### 使用函数进行部分格式化
这种用例适用于您知道总是希望以通用方式获取某个变量的情况。一个很好的例子是日期或时间。想象一下，您有一个提示词，并且总是希望它包含当前日期。您无法将其硬编码到提示词中，并且将其与其他输入变量一起传递也很不方便。在这种情况下，能够用一个总是返回当前日期的函数来部分格式化提示词会很方便。

In [8]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 10/27/2025, 08:40:04


也可以直接用部分格式化后的变量初始化提示词

In [ ]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))